In [5]:
import csv
import datetime
import h5py
import itertools
import numpy as np
import os
import pandas as pd
import sklearn.metrics
import sys
import time

sys.path.append("../src")
import localmodule


for aug_kind_str in ["none", "all"]:

    # Define constants.
    data_dir = localmodule.get_data_dir()
    dataset_name = localmodule.get_dataset_name()
    folds = localmodule.fold_units()
    models_dir = localmodule.get_models_dir()
    units = localmodule.get_units()
    n_units = len(units)
    model_name = "icassp-convnet"
    if not aug_kind_str == "none":
        model_name = "_".join(
            [model_name, "aug-" + aug_kind_str])
    model_dir = os.path.join(models_dir, model_name)

    # Initialize dict of best trials.
    adhoc_best_trials_dict = {}
    
    # Initialize "best-5" matrices.
    adhoc_val_best5_tps = np.zeros((n_units, 5))
    adhoc_val_best5_fps = np.zeros((n_units, 5))
    adhoc_val_best5_tns = np.zeros((n_units, 5))
    adhoc_val_best5_fns = np.zeros((n_units, 5))

    adhoc_test_best5_tps = np.zeros((n_units, 5))
    adhoc_test_best5_fps = np.zeros((n_units, 5))
    adhoc_test_best5_tns = np.zeros((n_units, 5))
    adhoc_test_best5_fns = np.zeros((n_units, 5))


    # Define list of trial combinations.
    list_of_trials = [list(range(5))] * n_units
    combinations = list(itertools.product(*list_of_trials))
    n_combinations = len(combinations)


    # Loop over recording units.
    for test_unit_id, test_unit_str in enumerate(units):

        # Define directory for test unit.
        unit_dir = os.path.join(model_dir, test_unit_str)

        # Create CSV file for metrics.
        metrics_name = "_".join([
            dataset_name,
            model_name,
            test_unit_str,
            "clip-metrics"
        ])
        metrics_path = os.path.join(unit_dir, metrics_name + ".csv")
        metrics_df = pd.read_csv(metrics_path)

        # Find 5 trials maximizing validation accuracy.
        adhoc_val_accs = np.array( \
            metrics_df["Ad hoc validation accuracy (%)"])
        adhoc_val_best5_trials = np.argsort(adhoc_val_accs)[::-1][:5]
        
        
        # Store best trial in dictionary.
        adhoc_val_best1_trial = adhoc_val_best5_trials[0]
        adhoc_best_trials_dict[test_unit_str] = adhoc_val_best1_trial

        # Read best 5 validation TP, FP, TN, and FN.
        adhoc_val_tps = np.array(metrics_df["Ad hoc validation TP"])
        adhoc_val_best5_tps[test_unit_id, :] = \
            adhoc_val_tps[adhoc_val_best5_trials]
        adhoc_val_fps = np.array(metrics_df["Ad hoc validation FP"])
        adhoc_val_best5_fps[test_unit_id, :] = \
            adhoc_val_fps[adhoc_val_best5_trials]
        adhoc_val_tns = np.array(metrics_df["Ad hoc validation TN"])
        adhoc_val_best5_tns[test_unit_id, :] = \
            adhoc_val_tns[adhoc_val_best5_trials]
        adhoc_val_fns = np.array(metrics_df["Ad hoc validation FN"])
        adhoc_val_best5_fns[test_unit_id, :] = \
            adhoc_val_fns[adhoc_val_best5_trials]

        # Read cv-best 5 test TP, FP, TN, and FN.
        adhoc_test_tps = np.array(metrics_df["Ad hoc test TP"])
        adhoc_test_best5_tps[test_unit_id, :] = \
            adhoc_test_tps[adhoc_val_best5_trials]
        adhoc_test_fps = np.array(metrics_df["Ad hoc test FP"])
        adhoc_test_best5_fps[test_unit_id, :] = \
            adhoc_test_fps[adhoc_val_best5_trials]
        adhoc_test_tns = np.array(metrics_df["Ad hoc test TN"])
        adhoc_test_best5_tns[test_unit_id, :] = \
            adhoc_test_tns[adhoc_val_best5_trials]
        adhoc_test_fns = np.array(metrics_df["Ad hoc test FN"])
        adhoc_test_best5_fns[test_unit_id, :] = \
            adhoc_test_fns[adhoc_val_best5_trials]


    # Get mean and standard deviation of validation accuracies.
    comb_best5_adhoc_val_tps = []
    comb_best5_adhoc_val_fps = []
    comb_best5_adhoc_val_tns = []
    comb_best5_adhoc_val_fns = []
    for comb_id in range(n_combinations):
        comb = list(combinations[comb_id])
        comb_best5_adhoc_val_tps.append(np.sum(np.array(
            [adhoc_val_best5_tps[t, comb[t]] for t in range(n_units)])))
        comb_best5_adhoc_val_fps.append(np.sum(np.array(
            [adhoc_val_best5_fps[t, comb[t]] for t in range(n_units)])))
        comb_best5_adhoc_val_tns.append(np.sum(np.array(
            [adhoc_val_best5_tns[t, comb[t]] for t in range(n_units)])))
        comb_best5_adhoc_val_fns.append(np.sum(np.array(
            [adhoc_val_best5_fns[t, comb[t]] for t in range(n_units)])))
    comb_best5_adhoc_val_tps = np.array(comb_best5_adhoc_val_tps)
    comb_best5_adhoc_val_fps = np.array(comb_best5_adhoc_val_fps)
    comb_best5_adhoc_val_tns = np.array(comb_best5_adhoc_val_tns)
    comb_best5_adhoc_val_fns = np.array(comb_best5_adhoc_val_fns)
    comb_best5_adhoc_val_accs =\
        (comb_best5_adhoc_val_tps+comb_best5_adhoc_val_tns)/\
        (comb_best5_adhoc_val_tps+comb_best5_adhoc_val_tns+\
         comb_best5_adhoc_val_fps+comb_best5_adhoc_val_fns)
    mean_best5_adhoc_val_accs = np.mean(comb_best5_adhoc_val_accs)
    std_best5_adhoc_val_accs = np.std(comb_best5_adhoc_val_accs)
    oracle_adhoc_val_accs = np.max(comb_best5_adhoc_val_accs)


    # Get mean and standard deviation of test accuracies.
    comb_best5_adhoc_test_tps = []
    comb_best5_adhoc_test_fps = []
    comb_best5_adhoc_test_tns = []
    comb_best5_adhoc_test_fns = []
    for comb_id in range(n_combinations):
        comb = list(combinations[comb_id])
        comb_best5_adhoc_test_tps.append(np.sum(np.array(
            [adhoc_test_best5_tps[t, comb[t]] for t in range(n_units)])))
        comb_best5_adhoc_test_fps.append(np.sum(np.array(
            [adhoc_test_best5_fps[t, comb[t]] for t in range(n_units)])))
        comb_best5_adhoc_test_tns.append(np.sum(np.array(
            [adhoc_test_best5_tns[t, comb[t]] for t in range(n_units)])))
        comb_best5_adhoc_test_fns.append(np.sum(np.array(
            [adhoc_test_best5_fns[t, comb[t]] for t in range(n_units)])))
    comb_best5_adhoc_test_tps = np.array(comb_best5_adhoc_test_tps)
    comb_best5_adhoc_test_fps = np.array(comb_best5_adhoc_test_fps)
    comb_best5_adhoc_test_tns = np.array(comb_best5_adhoc_test_tns)
    comb_best5_adhoc_test_fns = np.array(comb_best5_adhoc_test_fns)
    comb_best5_adhoc_test_accs =\
        (comb_best5_adhoc_test_tps+comb_best5_adhoc_test_tns)/\
        (comb_best5_adhoc_test_tps+comb_best5_adhoc_test_tns+\
         comb_best5_adhoc_test_fps+comb_best5_adhoc_test_fns)
    mean_best5_adhoc_test_accs = np.mean(comb_best5_adhoc_test_accs)
    std_best5_adhoc_test_accs = np.std(comb_best5_adhoc_test_accs)
    oracle_adhoc_test_accs = np.max(comb_best5_adhoc_test_accs)
    
    print("Augmentation: {}".format(aug_kind_str))
    print(adhoc_best_trials_dict)
    print("Validation accuracy: {:5.3f}% ± {:5.3f} (best 5)".format(
        mean_best5_adhoc_val_accs * 100, std_best5_adhoc_val_accs * 100))
    print("Validation accuracy: {:5.3f}% (oracle)".format(
        oracle_adhoc_val_accs * 100))
    print("Test accuracy: {:5.3f}% ± {:5.3f} (best 5)".format(
        mean_best5_adhoc_test_accs * 100, std_best5_adhoc_test_accs * 100))
    print("Test accuracy: {:5.3f}% (oracle)".format(
        oracle_adhoc_test_accs * 100))
    print("")

Augmentation: none
{'unit01': 5, 'unit02': 5, 'unit03': 3, 'unit05': 5, 'unit07': 5, 'unit10': 8}
Validation accuracy: 85.894% ± 7.108 (best 5)
Validation accuracy: 95.000% (oracle)
Test accuracy: 83.252% ± 5.294 (best 5)
Test accuracy: 92.821% (oracle)

Augmentation: all
{'unit01': 5, 'unit02': 7, 'unit03': 9, 'unit05': 7, 'unit07': 7, 'unit10': 8}
Validation accuracy: 94.928% ± 0.391 (best 5)
Validation accuracy: 96.091% (oracle)
Test accuracy: 94.852% ± 0.983 (best 5)
Test accuracy: 96.383% (oracle)



In [3]:
metrics_df

,Dataset,Augmentation kind,Test unit,Trial,Ad hoc threshold,Ad hoc validation TP,Ad hoc validation FP,Ad hoc validation TN,Ad hoc validation FN,Ad hoc validation accuracy (%),...,Cross-validated test TN,Cross-validated test FN,Oracle threshold,Test oracle TP,Test oracle FP,Test oracle TN,Test oracle FN,Test oracle accuracy (%),Validation AUC,Test AUC
0,BirdVox-70k,all,unit10,trial-0,0.5,11034,760,11509,285,95.570,...,6657,182,0.763477,6707,132,6627,212,97.485,99.075,99.100
1,BirdVox-70k,all,unit10,trial-1,0.5,11170,624,11275,519,95.154,...,6674,165,0.731720,6687,152,6536,303,96.673,98.660,98.623
2,BirdVox-70k,all,unit10,trial-2,0.5,10726,1068,11557,237,94.468,...,6711,128,0.715165,6709,130,6713,126,98.128,99.265,99.341
3,BirdVox-70k,all,unit10,trial-3,0.5,11118,676,11588,206,96.261,...,6700,139,0.807082,6714,125,6675,164,97.887,99.427,99.238
4,BirdVox-70k,all,unit10,trial-4,0.5,11265,529,11407,387,96.117,...,6665,174,0.725913,6624,215,6585,254,96.571,99.187,99.058
5,BirdVox-70k,all,unit10,trial-5,0.5,10996,798,11433,361,95.086,...,6723,116,0.910962,6723,116,6623,216,97.573,98.743,99.061
6,BirdVox-70k,all,unit10,trial-6,0.5,10797,997,11434,360,94.247,...,6606,233,0.781604,6721,118,6540,299,96.951,98.765,98.457
7,BirdVox-70k,all,unit10,trial-7,0.5,10679,1115,11522,272,94.120,...,6633,206,0.927408,6749,90,6486,353,96.761,98.867,98.440
8,BirdVox-70k,all,unit10,trial-8,0.5,11339,455,11454,340,96.630,...,6715,124,0.698886,6657,182,6644,195,97.244,99.180,99.177
9,BirdVox-70k,all,unit10,trial-9,0.5,11440,354,11342,452,96.583,...,6695,144,0.726714,6732,107,6613,226,97.565,99.166,99.024
